In [11]:
from LightningModules import AutoEncoder, FUCCIDataModule, ReconstructionVisualization
from kornia import tensor_to_image
from microfilm.colorify import multichannel_to_rgb
import numpy as np
from PIL import Image
from pathlib import Path
import torch

In [12]:
FUCCI_PATH = '/home/ishang/implicit-cell-model/FUCCI-dev-data'
LOGS = '/data/ishang/fucci_vae'
!ls {LOGS}

FUCCI_reference_VAE_2023_16_05_06_31  FUCCI_reference_VAE_2023_17_05_02_42
FUCCI_reference_VAE_2023_16_05_06_32  FUCCI_reference_VAE_2023_17_05_02_59
FUCCI_reference_VAE_2023_16_05_06_38  FUCCI_reference_VAE_2023_17_05_03_10
FUCCI_reference_VAE_2023_16_05_06_39  FUCCI_reference_VAE_2023_17_05_03_11
FUCCI_reference_VAE_2023_16_05_06_41  FUCCI_reference_VAE_2023_17_05_04_32
FUCCI_reference_VAE_2023_16_05_06_42  FUCCI_reference_VAE_2023_17_05_04_33
FUCCI_reference_VAE_2023_16_05_07_19  FUCCI_reference_VAE_2023_17_05_05_21
FUCCI_reference_VAE_2023_16_05_07_22  FUCCI_reference_VAE_2023_17_05_05_22
FUCCI_reference_VAE_2023_16_05_08_39  FUCCI_reference_VAE_2023_17_05_19_22
FUCCI_reference_VAE_2023_16_05_08_54  FUCCI_reference_VAE_2023_17_05_19_28
FUCCI_reference_VAE_2023_16_05_08_55  FUCCI_reference_VAE_2023_17_05_19_29
FUCCI_reference_VAE_2023_16_05_09_01  FUCCI_reference_VAE_2023_17_05_19_30
FUCCI_reference_VAE_2023_16_05_09_02  FUCCI_reference_VAE_2023_17_05_19_32
FUCCI_reference_VAE_2023_

In [13]:
model_folder = 'FUCCI_reference_VAE_2023_16_05_10_26'
!ls {LOGS}/{model_folder}/lightning_logs/

'epoch=99-Val_loss=0.00.ckpt'


In [14]:
checkpoint = 'epoch=99-Val_loss=0.00.ckpt'
model_path = Path(LOGS) / model_folder / "lightning_logs" / checkpoint
res = 64

In [15]:
# ckpt = torch.load(model_path)

In [16]:
# len(ckpt['state_dict'].keys())
# print(ckpt['state_dict'].keys())
# ckpt['state_dict']['decoder.layers.0.0.weight'].shape
# for k, v in ckpt['state_dict'].items():
#     print(k, v.shape)

In [21]:
import wandb
run = wandb.init()
artifact = run.use_artifact('lundberg-lab/FUCCI_reference_VAE/model-4pyjiycg:v0', type='model')
artifact_dir = artifact.download()
print(artifact_dir)
print(type(artifact_dir))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ishangaur (lundberg-lab). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-4pyjiycg:v0, 4657.23MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:12.6


./artifacts/model-4pyjiycg:v0
<class 'str'>


In [36]:
# !ls {artifact_dir}
model_path = Path(artifact_dir) / "model.ckpt"
print(model_path)
model = torch.load(model_path)
# model = AutoEncoder.load_from_checkpoint(model_path)

artifacts/model-4pyjiycg:v0/model.ckpt


In [37]:
dm = FUCCIDataModule(
    data_dir=FUCCI_PATH,
    dataset="reference",
    imsize=res,
    split=(0.8, 0.1, 0.1),
    batch_size=32,
    num_workers=8
)

print(len(dm.dataset))
print(type(dm.dataset))

32
<class 'FUCCIDataset.ReferenceChannelDatasetInMemory'>


In [39]:
# print(dm.dataset[:3].shape)
x = dm.dataset[:3]
x_hat = model(x)
print(x.shape)
print(x_hat.shape)
# x_hat[:, 0] *= 10
# x_hat = torch.ones_like(x) * torch.max(x_hat)

In [20]:
x = dm.dataset[:3]
grid = ReconstructionVisualization.make_reconstruction_grid(x, x)

shape = grid.shape
for i in range(3):
    for j in range(2):
        y_st, y_end = i * shape[1] // 3, (i + 1) * shape[1] // 3
        x_st, x_end = j * shape[2] // 2, (j + 1) * shape[2] // 2
        for c in range(2):
            print(torch.min(grid[c, y_st:y_end, x_st:x_end]), torch.max(grid[c, y_st:y_end, x_st:x_end]))
    print()

print(grid.shape)
# img = np.moveaxis(tensor_to_image(grid), -1, 0)
img = grid.cpu().detach().numpy()
print(img.shape)
for i in range(2):
    image_composite, _, _, _= multichannel_to_rgb(img[i], cmaps=[dm.dataset.channel_colors()[i]])
    image_composite = (255 * image_composite).astype(np.uint8)
    image_composite = Image.fromarray(image_composite)
    image_composite.save(f'pred_grid_{res}_{i}_composite.png')
image_composite, _, _, _= multichannel_to_rgb(img, cmaps=dm.dataset.channel_colors())
image_composite = (255 * image_composite).astype(np.uint8)
image_composite = Image.fromarray(image_composite)
image_composite.save(f'pred_grid_{res}_composite.png')

data_grid = ReconstructionVisualization.make_reconstruction_grid(x, x)
print(data_grid.shape)
data_img = np.moveaxis(tensor_to_image(data_grid), -1, 0)
print(data_img.shape)
data_composite, _, _, _= multichannel_to_rgb(data_img, cmaps=dm.dataset.channel_colors())
data_composite = (255 * data_composite).astype(np.uint8)
data_composite = Image.fromarray(data_composite)
data_composite.save(f'data_grid_{res}_composite.png')

RuntimeError: stack expects each tensor to be equal size, but got [3, 2, 256, 256] at entry 0 and [48, 2, 64, 64] at entry 1